In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# Cargar grafo
pb_path = 'DS_CNN_S.pb'

with tf.gfile.GFile(pb_path, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

# Importar grafo y crear sesión
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name='')

    sess = tf.Session(graph=graph)

    # Inspeccionamos nombres de tensores
    for op in graph.get_operations():
        print(op.name)


wav_data
decoded_sample_data
AudioSpectrogram
Mfcc
Reshape/shape
Reshape
Reshape_1/shape
Reshape_1
MobileNet/conv_1/weights
MobileNet/conv_1/weights/read
MobileNet/conv_1/biases
MobileNet/conv_1/biases/read
MobileNet/conv_1/Conv2D
MobileNet/conv_1/BiasAdd
MobileNet/conv_1/batch_norm/beta
MobileNet/conv_1/batch_norm/beta/read
MobileNet/conv_1/batch_norm/Const
MobileNet/conv_1/batch_norm/moving_mean
MobileNet/conv_1/batch_norm/moving_mean/read
MobileNet/conv_1/batch_norm/moving_variance
MobileNet/conv_1/batch_norm/moving_variance/read
MobileNet/conv_1/batch_norm/FusedBatchNorm
MobileNet/conv_1/batch_norm/Relu
MobileNet/conv_ds_1/depthwise_conv/depthwise_weights
MobileNet/conv_ds_1/depthwise_conv/depthwise_weights/read
MobileNet/conv_ds_1/depthwise_conv/depthwise
MobileNet/conv_ds_1/depthwise_conv/biases
MobileNet/conv_ds_1/depthwise_conv/biases/read
MobileNet/conv_ds_1/depthwise_conv/BiasAdd
MobileNet/conv_ds_1/dw_batch_norm/beta
MobileNet/conv_ds_1/dw_batch_norm/beta/read
MobileNet/conv

In [4]:
import librosa
import numpy as np

def preprocess_audio(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    y = y[:16000] if len(y) > 16000 else np.pad(y, (0, max(0, 16000 - len(y))))  # 1 segundo

    mel_spec = librosa.feature.melspectrogram(y=y, sr=16000, n_fft=640, hop_length=320,
                                              n_mels=40, fmin=20, fmax=4000, center=False)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spec), n_mfcc=10)
    mfcc = mfcc.T  # (49, 10)
    mfcc = np.expand_dims(mfcc, axis=-1)       # (49, 10, 1)
    mfcc = np.expand_dims(mfcc, axis=0)        # (1, 49, 10, 1)

    return mfcc.astype(np.float32)


In [9]:
# Cargar audio
mfcc = preprocess_audio("right.wav")

# Obtener tensores
input_tensor = graph.get_tensor_by_name('Reshape_1:0')
output_tensor = graph.get_tensor_by_name('labels_softmax:0')

# Ejecutar inferencia
output = sess.run(output_tensor, feed_dict={input_tensor: mfcc})

# Etiquetas (mismo orden del repo)
labels = ["silence", "unknown", "yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

print("Predicción:", labels[np.argmax(output)])
print("Probabilidades:", output)


Predicción: right
Probabilidades: [[3.1013939e-10 1.1702927e-03 6.9647067e-06 1.2431881e-02 6.6332422e-07
  4.7662769e-07 1.6167814e-05 9.6144825e-01 3.3294305e-06 1.2067918e-07
  2.3447440e-08 2.4921818e-02]]


Hasta acá el modelo sin cuantizar funciona perfecto

# Cuantización

In [10]:
import tensorflow as tf
print(tf.__version__)  # debería ser 2.x


2.18.0


In [19]:
import tensorflow as tf

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='DS_CNN_S.pb',
    input_arrays=['Reshape_1'],             # nombre del tensor de entrada
    output_arrays=['labels_softmax'],       # nombre del tensor de salida
    input_shapes={'Reshape_1': [1, 49, 10, 1]}
)

# Sin cuantización
tflite_model = converter.convert()

# Guardar el modelo
with open('ds_cnn_float.tflite', 'wb') as f:
    f.write(tflite_model)

print("Modelo convertido a float TFLite exitosamente.")



Modelo convertido a float TFLite exitosamente.


In [ ]:
import os
import numpy as np
import tensorflow as tf
import librosa

def standardize_preprocessing_REVISED(file_path, sr=16000, n_mfcc=10, n_fft=640, hop_length=320, n_mels=40, fmin=20, fmax=4000):
    """
    Standardized audio preprocessing to generate MFCCs.
    Matches the original successful .pb model preprocessing from cell 2.
    Explicit per-sample MFCC normalization is REMOVED.
    """
    y, sr_loaded = librosa.load(file_path, sr=sr)

    # Pad/truncate to 1 second (16000 samples)
    if len(y) > sr:
        y = y[:sr]
    else:
        y = np.pad(y, (0, max(0, sr - len(y))), 'constant')

    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length,
                                              n_mels=n_mels, fmin=fmin, fmax=fmax, center=False) #
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spec), n_mfcc=n_mfcc) #
    mfcc = mfcc.T  # Transpose to (time_frames, n_mfcc) which is (49, 10)


    if mfcc.shape[0] != 49:
        if mfcc.shape[0] < 49:
            pad_width = 49 - mfcc.shape[0]
            mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
        elif mfcc.shape[0] > 49:
            mfcc = mfcc[:49, :]

    # Reshape for TFLite model: (1, 49, 10, 1)
    mfcc = np.expand_dims(mfcc, axis=-1) # (49, 10, 1)
    mfcc = np.expand_dims(mfcc, axis=0)  # (1, 49, 10, 1)

    return mfcc.astype(np.float32)


In [177]:
import tensorflow as tf # Make sure TF is imported if not already
import os # For file checks

# Assuming ds_cnn_float.tflite was generated in cell 19
model_path_float = 'ds_cnn_float.tflite'
if not os.path.exists(model_path_float):
    print(f"Model file not found: {model_path_float}")
else:
    interpreter_float = tf.lite.Interpreter(model_path=model_path_float)
    interpreter_float.allocate_tensors()

    input_details_float = interpreter_float.get_input_details()
    output_details_float = interpreter_float.get_output_details()

    # Prepare input using the REVISED standardized function
    audio_file = "down.wav" # Use one of your test files like "right.wav"
    if not os.path.exists(audio_file):
        print(f"Audio file not found: {audio_file}. Please make sure it exists.")
        # You might need to upload it to your Colab environment if it's not there
        # e.g. by running:
        # !wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02/right/00f0204f_nohash_0.wav -O right.wav
        # (or use any other .wav file you have that worked with the .pb model)
    else:
        input_data_float = standardize_preprocessing_REVISED(audio_file)

        interpreter_float.set_tensor(input_details_float[0]['index'], input_data_float)
        interpreter_float.invoke()
        output_data_float = interpreter_float.get_tensor(output_details_float[0]['index'])

        labels = ["silence", "unknown", "yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"] #
        print(f"--- Float TFLite Model ({audio_file}) with REVISED Preprocessing ---")
        print("Input data shape:", input_data_float.shape)
        print("Input data type:", input_data_float.dtype)
        print("Prediction:", labels[np.argmax(output_data_float)])
        print("Probabilities:", output_data_float)

--- Float TFLite Model (down.wav) with REVISED Preprocessing ---
Input data shape: (1, 49, 10, 1)
Input data type: float32
Prediction: down
Probabilities: [[1.2286399e-08 8.7782582e-03 1.3783874e-04 3.2073833e-02 1.1164533e-06
  9.5302641e-01 3.5775043e-05 1.3056492e-03 2.4862279e-04 8.4167652e-07
  1.4483442e-03 2.9433304e-03]]


Hasta aqui, el modelo float puede predecir bien

In [ ]:
import os
import librosa
import numpy as np

AUDIO_DIR = '/content/representative_audio_files/'
if not os.path.exists(AUDIO_DIR):
    os.makedirs(AUDIO_DIR)
    print(f"Created directory: {AUDIO_DIR}")
    print(f"Please upload your representative wav files to this directory.")

def representative_data_gen_REVISED():
    print(f"Looking for audio files in: {AUDIO_DIR}")
    file_list = [f for f in os.listdir(AUDIO_DIR) if f.endswith('.wav')]
    if not file_list:
        print(f"No .wav files found in {AUDIO_DIR}. Representative dataset will be empty.")
        return

    count = 0
    num_samples_to_yield = min(len(file_list), 150) # Use up to 150 samples or all if fewer

    for fname in file_list:
        audio_path = os.path.join(AUDIO_DIR, fname)
        try:
            mfcc_float = standardize_preprocessing_REVISED(audio_path)
            yield [mfcc_float]
            count += 1
            if count >= num_samples_to_yield:
                print(f"Generated {count} representative samples.")
                break
        except Exception as e:
            print(f"Skipping {audio_path} due to error during preprocessing: {e}")

    if count == 0:
        print("No representative audio files were successfully processed. Quantization will likely fail or be poor.")
    elif count < 100 and len(file_list) >=100 : # if many files available but few processed
         print(f"Warning: Only {count} representative samples processed successfully. Check for errors.")
    elif count < 50 : # if few files available or processed
        print(f"Warning: Only {count} representative samples. Consider adding more (100-300 recommended) for better quantization.")

In [ ]:
import tensorflow as tf # Ensure tf is imported

pb_file_path = 'DS_CNN_S.pb'
if not os.path.exists(pb_file_path):
    print(f"Error: Frozen graph file {pb_file_path} not found. Please ensure it's in the correct location.")
else:
    converter_int8_revised = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
        graph_def_file=pb_file_path,
        input_arrays=['Reshape_1'], #
        output_arrays=['labels_softmax'], #
        input_shapes={'Reshape_1': [1, 49, 10, 1]} #
    )

    converter_int8_revised.optimizations = [tf.lite.Optimize.DEFAULT]
    converter_int8_revised.representative_dataset = representative_data_gen_REVISED
    converter_int8_revised.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter_int8_revised.inference_input_type = tf.uint8 # o tf.int8
    converter_int8_revised.inference_output_type = tf.uint8 # o tf.int8

    tflite_quant_model_revised = converter_int8_revised.convert()

    quantized_model_path = 'ds_cnn_int8_revised.tflite'
    with open(quantized_model_path, 'wb') as f:
        f.write(tflite_quant_model_revised)
    print(f"Revised model quantized to int8 saved as: {quantized_model_path}")

Looking for audio files in: /content/representative_audio_files/
Generated 120 representative samples.
Revised model quantized to int8 saved as: ds_cnn_int8_revised.tflite


In [ ]:
# Load the new revised int8 model
interpreter_int8_revised = tf.lite.Interpreter(model_path='ds_cnn_int8_revised.tflite')
interpreter_int8_revised.allocate_tensors()

input_details_int8_revised = interpreter_int8_revised.get_input_details()
output_details_int8_revised = interpreter_int8_revised.get_output_details()

print("Revised Int8 Input Details:", input_details_int8_revised)
print("Revised Int8 Output Details:", output_details_int8_revised)

input_scale, input_zero_point = input_details_int8_revised[0]['quantization']

def preprocess_audio_for_int8_revised_inference(audio_path):
    mfcc_float = standardize_preprocessing_REVISED(audio_path)

    # 2. Quantize the float input to match model's expected input type
    mfcc_quantized = (mfcc_float / input_scale) + input_zero_point
    mfcc_quantized = mfcc_quantized.astype(input_details_int8_revised[0]['dtype'])
    return mfcc_quantized

# Test with multiple audio files
labels = ["silence", "unknown", "yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"] #
audio_files_to_test = { 
    'yes': 'yes.wav',
    'no': 'no.wav',
    'right': 'right.wav',
}

print("\n--- REVISED INT8 Quantized Model Predictions ---")
for label_key, path in audio_files_to_test.items():
    if not os.path.exists(path):
        print(f"Warning: Audio file for testing '{label_key}' ({path}) not found. Skipping.")
        continue

    input_data_int8 = preprocess_audio_for_int8_revised_inference(path)
    interpreter_int8_revised.set_tensor(input_details_int8_revised[0]['index'], input_data_int8)
    interpreter_int8_revised.invoke()
    output_data_int8 = interpreter_int8_revised.get_tensor(output_details_int8_revised[0]['index'])

    pred_index = np.argmax(output_data_int8[0])
    print(f"Audio: {label_key} ({path})")
    print(f"Prediction: {labels[pred_index]}")
    print(f"Raw Output Scores (quantized): {output_data_int8[0]}\n")

Revised Int8 Input Details: [{'name': 'Reshape_1', 'index': 0, 'shape': array([ 1, 49, 10,  1], dtype=int32), 'shape_signature': array([ 1, 49, 10,  1], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (2.602257490158081, 198), 'quantization_parameters': {'scales': array([2.6022575], dtype=float32), 'zero_points': array([198], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Revised Int8 Output Details: [{'name': 'labels_softmax', 'index': 34, 'shape': array([ 1, 12], dtype=int32), 'shape_signature': array([ 1, 12], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

--- REVISED INT8 Quantized Model Predictions ---
Audio: yes (yes.wav)
Prediction: yes
Raw Output Scores (quantized): [  0   0 255   0   0   0   0   0   0   0   0   0]

Audio: no (no.wav)
Predictio

In [ ]:

def preprocess_audio(path):
    # The dynamic range quantized model expects float32 input.
    return standardize_preprocessing_REVISED(path)



In [ ]:
def standardize_preprocessing_from_data_REVISED(y, sr_loaded, target_sr=16000, n_mfcc=10, n_fft=640, hop_length=320, n_mels=40, fmin=20, fmax=4000):
    if sr_loaded != target_sr:
        # Ensure y is float before resampling if it's not already
        y = y.astype(np.float32) if not np.issubdtype(y.dtype, np.floating) else y
        y = librosa.resample(y, orig_sr=sr_loaded, target_sr=target_sr)

    # Pad/truncate to 1 second (target_sr samples)
    if len(y) > target_sr:
        y = y[:target_sr]
    else:
        y = np.pad(y, (0, max(0, target_sr - len(y))), 'constant')

    mel_spec = librosa.feature.melspectrogram(y=y, sr=target_sr, n_fft=n_fft, hop_length=hop_length,
                                              n_mels=n_mels, fmin=fmin, fmax=fmax, center=False)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spec), n_mfcc=n_mfcc)
    mfcc = mfcc.T

    if mfcc.shape[0] != 49:
        if mfcc.shape[0] < 49:
            pad_width = 49 - mfcc.shape[0]
            mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
        elif mfcc.shape[0] > 49:
            mfcc = mfcc[:49, :]

    # NO explicit MFCC normalization
    mfcc = np.expand_dims(mfcc, axis=-1)
    mfcc = np.expand_dims(mfcc, axis=0)
    return mfcc.astype(np.float32)



In [197]:
from IPython.display import display, HTML
from google.colab import output
from base64 import b64decode
import io
import numpy as np
import ffmpeg
from scipy.io.wavfile import read as wav_read

AUDIO_HTML = """
<script>
var recordButton = document.createElement("BUTTON");
recordButton.innerText = "Start Recording";
document.body.appendChild(recordButton);

var recorder;
var gumStream;
var audioChunks = [];

recordButton.onclick = async () => {
  if (recorder && recorder.state == "recording") {
    recorder.stop();
    gumStream.getAudioTracks()[0].stop();
    recordButton.innerText = "Start Recording";
  } else {
    audioChunks = [];
    let stream = await navigator.mediaDevices.getUserMedia({audio:true});
    gumStream = stream;
    recorder = new MediaRecorder(stream);
    recorder.ondataavailable = e => audioChunks.push(e.data);
    recorder.onstop = e => {
      let blob = new Blob(audioChunks, {type: 'audio/webm'});
      var reader = new FileReader();
      reader.readAsDataURL(blob);
      reader.onloadend = () => {
        var base64data = reader.result;
        google.colab.kernel.invokeFunction('notebook.receive_audio', [base64data], {});
      };
    };
    recorder.start();
    recordButton.innerText = "Stop Recording";
  }
};
</script>
"""

audio_bytes = None

def receive_audio(data):
    global audio_bytes
    header, encoded = data.split(',', 1)
    audio_bytes = b64decode(encoded)

output.register_callback('notebook.receive_audio', receive_audio)

def record_audio():
    display(HTML(AUDIO_HTML))

def process_audio(audio_bytes):
    process = (
        ffmpeg
        .input('pipe:0')
        .output('pipe:1', format='wav', ac=1, ar='16000')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )
    output_wav, _ = process.communicate(input=audio_bytes)
    sr, audio = wav_read(io.BytesIO(output_wav))
    return audio, sr

print("Ejecuta record_audio() para grabar. Luego usa process_audio(audio_bytes) para convertirlo y hacer inferencia.")



Ejecuta record_audio() para grabar. Luego usa process_audio(audio_bytes) para convertirlo y hacer inferencia.


In [183]:
import numpy as np
import tensorflow as tf
import librosa # Make sure librosa is imported
from python_speech_features import mfcc
from IPython.display import Audio


def run_tflite_inference_singleFile(model_path, audio_data, sample_rate):
    """
    Runs inference on a TFLite model with a single audio input.
    Handles both float and quantized (int8/uint8) models.

    Args:
        model_path (str): Path to the .tflite model file.
        audio_data (np.array): Raw audio data.
        sample_rate (int): Sample rate of the audio_data.

    Returns:
        tuple: (predicted_label_index, predicted_label, probabilities_or_scores)
               probabilities_or_scores will be float probabilities if dequantized,
               otherwise raw quantized scores.
    """
    labels = ['silence', 'unknown', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go'] #

    # Load TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Preprocess audio data to get float MFCCs
    # This uses the REVISED logic (no extra MFCC normalization)
    input_mfcc_float = standardize_preprocessing_from_data_REVISED(audio_data, sample_rate)

    # Check if model is quantized and quantize input if necessary
    input_tensor_data = input_mfcc_float
    if input_details['dtype'] == np.uint8 or input_details['dtype'] == np.int8:
        input_scale, input_zero_point = input_details['quantization']
        if input_scale != 0.0 or input_zero_point != 0: # Check if valid quantization params
             input_tensor_data = (input_mfcc_float / input_scale) + input_zero_point
             input_tensor_data = input_tensor_data.astype(input_details['dtype'])
        else: # Should not happen for properly quantized models with int8/uint8 type
            print("Warning: Model input is int8/uint8 but quantization params are trivial. Using float input.")
            # Or, one might decide to cast if the model truly expects unscaled int values,
            # but that's unusual for TFLite converted models.
            # Forcing it to float might cause errors if it strictly expects int.
            # input_tensor_data = input_mfcc_float.astype(input_details['dtype']) # Risky if no scale/zp

    # Set input tensor
    # Ensure input_tensor_data shape matches model's expected input shape,
    # standardize_preprocessing_from_data_REVISED should already produce the correct shape [1, 49, 10, 1]
    if tuple(input_tensor_data.shape) != tuple(input_details['shape']):
        print(f"Warning: Reshaping input from {input_tensor_data.shape} to {input_details['shape']}")
        input_tensor_data = np.reshape(input_tensor_data, input_details['shape'])

    interpreter.set_tensor(input_details['index'], input_tensor_data)

    # Run inference
    interpreter.invoke()

    # Get output tensor
    output_data = interpreter.get_tensor(output_details['index'])

    output_scores = output_data[0] # Assuming batch size of 1

    # Dequantize output if necessary to get float probabilities
    if output_details['dtype'] == np.uint8 or output_details['dtype'] == np.int8:
        output_scale, output_zero_point = output_details['quantization']
        if output_scale != 0.0 or output_zero_point != 0:
            output_scores_float = (output_scores.astype(np.float32) - output_zero_point) * output_scale
            probabilities_or_scores = output_scores_float
        else: # Output is quantized but no dequantization params, return raw scores
            probabilities_or_scores = output_scores
            print("Warning: Output is quantized but dequantization params are trivial. Returning raw scores.")
    else: # Output is already float
        probabilities_or_scores = output_scores

    # Get prediction
    pred_idx = np.argmax(probabilities_or_scores) # Argmax on float probabilities or raw scores
    predicted_label = labels[pred_idx]

    print(f"\nModel: {model_path}")
    print(f"Predicted Index: {pred_idx}")
    print(f"Predicted Label: {predicted_label}")
    if output_details['dtype'] == np.float32:
        print(f"Probabilities: {np.round(probabilities_or_scores, 4)}")
    else:
        print(f"Dequantized Scores (if applicable, otherwise raw): {np.round(probabilities_or_scores, 4)}")
        print(f"Raw Quantized Output Scores: {output_data[0]}")


    return pred_idx, predicted_label, probabilities_or_scores

In [198]:
record_audio()

In [214]:
audio, sr = process_audio(audio_bytes)
Audio(data=audio, rate=sr)

<ipython-input-197-1d6e0a5c5b76>:66: WavFileWarning: Reached EOF prematurely; finished at 32719 bytes, expected 4294967303 bytes from header.
  sr, audio = wav_read(io.BytesIO(output_wav))


In [215]:
from scipy.io.wavfile import write
write("test.wav", sr, audio)

In [216]:
file_to_test = "test.wav"
audio, sr = librosa.load(file_to_test, sr=16000)

In [199]:
# Or load from a file
#file_to_test = "off.wav"
#audio, sr = librosa.load(file_to_test, sr=16000)
#Audio(data=audio, rate=sr)

<ipython-input-197-1d6e0a5c5b76>:66: WavFileWarning: Reached EOF prematurely; finished at 46159 bytes, expected 4294967303 bytes from header.
  sr, audio = wav_read(io.BytesIO(output_wav))


In [ ]:
# Example with the float model
model_path_to_test = 'ds_cnn_float.tflite'
pred_idx, label, probs = run_tflite_inference_singleFile(model_path_to_test, audio, sr)

# Example with the revised int8 quantized model
model_path_to_test_int8 = 'ds_cnn_int8_revised.tflite'
pred_idx_int8, label_int8, scores_int8 = run_tflite_inference_singleFile(model_path_to_test_int8, audio, sr)

# Example with the dynamic range quantized model
model_path_to_test_dynamic = 'ds_cnn_dynamic.tflite'
pred_idx_dyn, label_dyn, probs_dyn = run_tflite_inference_singleFile(model_path_to_test_dynamic, audio, sr)


Model: ds_cnn_float.tflite
Predicted Index: 9
Predicted Label: off
Probabilities: [0.000e+00 1.000e-04 0.000e+00 2.000e-04 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 9.981e-01 0.000e+00 1.500e-03]

Model: ds_cnn_int8_revised.tflite
Predicted Index: 9
Predicted Label: off
Dequantized Scores (if applicable, otherwise raw): [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.9844
 0.     0.0117]
Raw Quantized Output Scores: [  0   0   0   0   0   0   0   0   0 252   0   3]

Model: ds_cnn_dynamic.tflite
Predicted Index: 9
Predicted Label: off
Probabilities: [0.000e+00 2.000e-04 0.000e+00 3.000e-04 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 9.976e-01 0.000e+00 1.800e-03]
